# Imports

In [1]:

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import warnings
import data_func.read_data as read_data
warnings.filterwarnings("ignore", category=FutureWarning, module="xgboost")
warnings.filterwarnings("ignore", category=FutureWarning, message="is_sparse is deprecated")


# Load datasets

In [2]:
dataframes = read_data.get_training_data()
X_frames_train = dataframes[0]
Y_frames_train = dataframes[1]
X_frames_test = read_data.get_test_data()


In [3]:
XTRA = X_frames_train[0]
XTRB = X_frames_train[1]
XTRC = X_frames_train[2]

YA = Y_frames_train[0]
YB = Y_frames_train[1]
YC = Y_frames_train[2]

In [4]:
print(XTRA["date_forecast"].head())
print(YA["time"].head())

0   2019-06-02 22:00:00
1   2019-06-02 22:15:00
2   2019-06-02 22:30:00
3   2019-06-02 22:45:00
4   2019-06-02 23:00:00
Name: date_forecast, dtype: datetime64[ns]
0   2019-06-02 22:00:00
1   2019-06-02 23:00:00
2   2019-06-03 00:00:00
3   2019-06-03 01:00:00
4   2019-06-03 02:00:00
Name: time, dtype: datetime64[ns]


To align we can drop entry 0 for both Y and X

In [5]:
print(XTRB["date_forecast"].head())
print(YB["time"].head())

0   2019-01-01 00:00:00
1   2019-01-01 00:15:00
2   2019-01-01 00:30:00
3   2019-01-01 00:45:00
4   2019-01-01 01:00:00
Name: date_forecast, dtype: datetime64[ns]
0   2018-12-31 23:00:00
1   2019-01-01 00:00:00
2   2019-01-01 01:00:00
3   2019-01-01 02:00:00
4   2019-01-01 03:00:00
Name: time, dtype: datetime64[ns]


To align we remove 0 and 1 from Y and 0 from X

In [6]:
print(XTRC["date_forecast"].head())
print(YC["time"].head())

0   2019-01-01 00:00:00
1   2019-01-01 00:15:00
2   2019-01-01 00:30:00
3   2019-01-01 00:45:00
4   2019-01-01 01:00:00
Name: date_forecast, dtype: datetime64[ns]
0   2018-12-31 23:00:00
1   2019-01-01 00:00:00
2   2019-01-01 01:00:00
3   2019-01-01 02:00:00
4   2019-01-01 03:00:00
Name: time, dtype: datetime64[ns]


To align we remove 0 and 1 from Y and 0 from X

In [7]:
XTRA = XTRA.drop([0]).reset_index(drop=True)
XTRB = XTRB.drop([0]).reset_index(drop=True)
XTRC = XTRC.drop([0]).reset_index(drop=True)

YA = YA.drop([0]).reset_index(drop=True)
YB = YB.drop([0, 1]).reset_index(drop=True)
YC = YC.drop([0, 1]).reset_index(drop=True)

Need to reset index for aggregation function

In [8]:
import data_func.aggregation as data_agg


In [9]:
for x in X_frames_test:
    print(x["date_forecast"].head())

0   2023-05-01 00:00:00
1   2023-05-01 00:15:00
2   2023-05-01 00:30:00
3   2023-05-01 00:45:00
4   2023-05-01 01:00:00
Name: date_forecast, dtype: datetime64[ns]
0   2023-05-01 00:00:00
1   2023-05-01 00:15:00
2   2023-05-01 00:30:00
3   2023-05-01 00:45:00
4   2023-05-01 01:00:00
Name: date_forecast, dtype: datetime64[ns]
0   2023-05-01 00:00:00
1   2023-05-01 00:15:00
2   2023-05-01 00:30:00
3   2023-05-01 00:45:00
4   2023-05-01 01:00:00
Name: date_forecast, dtype: datetime64[ns]


drop 0 since we want to match on the "whole" hour.

In [10]:
for i in range(len(X_frames_test)):
    X_frames_test[i] = X_frames_test[i].drop([0]).reset_index(drop=True)
    X_frames_test[i] = X_frames_test[i].drop(columns=["date_calc"])
    print(X_frames_test[i]["date_forecast"].head())

0   2023-05-01 00:15:00
1   2023-05-01 00:30:00
2   2023-05-01 00:45:00
3   2023-05-01 01:00:00
4   2023-05-01 01:15:00
Name: date_forecast, dtype: datetime64[ns]
0   2023-05-01 00:15:00
1   2023-05-01 00:30:00
2   2023-05-01 00:45:00
3   2023-05-01 01:00:00
4   2023-05-01 01:15:00
Name: date_forecast, dtype: datetime64[ns]
0   2023-05-01 00:15:00
1   2023-05-01 00:30:00
2   2023-05-01 00:45:00
3   2023-05-01 01:00:00
4   2023-05-01 01:15:00
Name: date_forecast, dtype: datetime64[ns]


# Data clean up

In [11]:
# making shure that target values line up with x_values
import data_func.aggregation as data_agg

categorical_col = ['dew_or_rime:idx', 'precip_type_5min:idx']

def aggregate_correct_x(x: pd.DataFrame) -> pd.DataFrame:
   categorical = x[["date_forecast"] + categorical_col]
   mean = x.drop(columns=categorical_col)

   categorical = data_agg.gen_agg(categorical, data_agg.stocastic_median)
   mean = data_agg.gen_agg(mean, "mean")

   return pd.merge(categorical, mean, on="date_forecast")

def data_allign(x_train, y_train):

  y_train.dropna(inplace=True)
  x_train = aggregate_correct_x(x_train)
  combined_data = pd.merge(x_train, y_train, left_on='date_forecast', right_on='time')
  y_train = combined_data['pv_measurement']

  if 'time' and 'pv_measurement' in combined_data.columns:
    combined_data.drop(columns=['time', 'pv_measurement'], inplace=True)
    
  return combined_data, y_train



X_train = [XTRA, XTRB, XTRC]
Y_train = [YA, YB, YC]

for i in range(len(X_train)):
    X_train[i], Y_train[i] = data_allign(X_train[i], Y_train[i])

for j in range(len(X_frames_test)):
    X_frames_test[j] = aggregate_correct_x(X_frames_test[j])




In [12]:
#Verify length matches
for x in range(len(X_train)):
    print("x,y: ", len(X_train[x]), len(Y_train[x]))



x,y:  34059 34059
x,y:  32817 32817
x,y:  26070 26070


# Feature engineering

In [13]:
import data_func.timeseasonality as DTS
import data_func.one_hot_encoding as OHE
for i in range(len(X_train)):
    X_train[i] = DTS.append_seasonal_columns(X_train[i])
    X_train[i].drop(columns=['date_forecast'], inplace=True)

for i in range(len(X_frames_test)):
    X_frames_test[i] = DTS.append_seasonal_columns(X_frames_test[i])
    X_frames_test[i].drop(columns=['date_forecast'], inplace=True)

# THIS SECTION CAN ONLY WORK IF THE PREVIOUS AGGREGATION IS DONE INDIVIDUALLY FOR CATEGORICAL DATA
# import data_func.one_hot_encoding as OHE

# for i in range(len(X_train)):
#      X_train[i] = OHE.one_hot_encode(X_train[i], ['dew_or_rime:idx', 'precip_type_5min:idx'])

# for i in range(len(X_frames_test)):
#      X_frames_test[i] = OHE.one_hot_encode(X_frames_test[i], ['dew_or_rime:idx', 'precip_type_5min:idx'])
#      X_frames_test[i]["dew_or_rime:idx_-1"] = 0 
#      X_frames_test[i]["precip_type_5min:idx_2"] = 0 
#      X_frames_test[i]["precip_type_5min:idx_3"] = 0 



In [14]:
X_train[0].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
dew_or_rime:idx,34059.0,0.018908,0.187693,-1.000000,0.000000,0.000000e+00,0.000000,1.000000e+00
precip_type_5min:idx,34059.0,0.073079,0.338962,0.000000,0.000000,0.000000e+00,0.000000,3.000000e+00
absolute_humidity_2m:gm3,34059.0,6.322364,2.799469,0.700000,4.200000,5.825000e+00,8.250000,1.740000e+01
air_density_2m:kgm3,34059.0,1.253278,0.037206,1.145000,1.227500,1.251500e+00,1.277250,1.426000e+00
ceiling_height_agl:m,27944.0,2968.545654,2557.649414,27.875000,1127.468750,1.979925e+03,4152.162598,1.229490e+04
clear_sky_energy_1h:J,34059.0,552703.625000,848353.312500,0.000000,0.000000,1.964755e+04,902072.968750,2.998650e+06
clear_sky_rad:W,34059.0,153.528702,236.248672,0.000000,0.000000,4.300000e+00,249.774994,8.338000e+02
cloud_base_agl:m,31705.0,1727.325317,1802.182373,27.799999,583.000000,1.147200e+03,2082.824951,1.166355e+04
dew_point_2m:K,34059.0,275.971863,6.892369,251.000000,271.299988,2.760000e+02,281.349976,2.937000e+02
diffuse_rad:W,34059.0,42.072437,62.838772,0.000000,0.000000,2.200000e+00,70.750000,3.355500e+02


In [15]:
X_frames_test[0].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
dew_or_rime:idx,720.0,3.194444e-02,1.759744e-01,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000e+00
precip_type_5min:idx,720.0,6.388889e-02,2.447249e-01,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000e+00
absolute_humidity_2m:gm3,720.0,8.207511e+00,2.174437e+00,3.200000,6.700000,8.075001e+00,1.000000e+01,1.360000e+01
air_density_2m:kgm3,720.0,1.233123e+00,3.207187e-02,1.151000,1.209000,1.238250e+00,1.260000e+00,1.300000e+00
ceiling_height_agl:m,539.0,3.094400e+03,2.935860e+03,53.299999,973.349976,1.682825e+03,4.138388e+03,1.141760e+04
clear_sky_energy_1h:J,720.0,1.227651e+06,1.101479e+06,0.000000,63704.814453,1.030980e+06,2.348519e+06,2.998053e+06
clear_sky_rad:W,720.0,3.410140e+02,3.069118e+02,0.000000,16.337499,2.793625e+02,6.492313e+02,8.334250e+02
cloud_base_agl:m,654.0,1.872781e+03,2.073476e+03,30.250000,513.025024,1.059325e+03,2.423094e+03,1.125670e+04
dew_point_2m:K,720.0,2.808040e+02,4.330719e+00,268.100006,278.024994,2.810625e+02,2.843500e+02,2.894500e+02
diffuse_rad:W,720.0,8.490232e+01,7.859216e+01,0.000000,8.500000,7.442500e+01,1.339812e+02,3.140500e+02


# Training the model

In [16]:
x_train_a, x_val_a, y_train_a, y_val_a = train_test_split(X_train[0], Y_train[0], test_size=0.17, random_state=None)
x_train_b, x_val_b, y_train_b, y_val_b = train_test_split(X_train[1], Y_train[1], test_size=0.17, random_state=None)
x_train_c, x_val_c, y_train_c, y_val_c = train_test_split(X_train[2], Y_train[2], test_size=0.17, random_state=None)

In [16]:
# Use params from hyperparameter tuning using optuna
params_a = {'random_state': 42, 'n_estimators': 980, 'max_depth': 9, 'learning_rate': 0.029035565559484028, 'subsample': 0.8393121619033767, 'colsample_bytree': 0.7589542758688459}
params_b = {'random_state': 42, 'n_estimators': 955, 'max_depth': 9, 'learning_rate': 0.02949625834198986, 'subsample': 0.8030196155828968, 'colsample_bytree': 0.72518389089994}
params_c = {'random_state': 42, 'n_estimators': 727, 'max_depth': 8, 'learning_rate': 0.08609213174337473, 'subsample': 0.8107057409889747, 'colsample_bytree': 0.8763563332327975}
model_a = xgb.XGBRegressor(**params_a)
model_b = xgb.XGBRegressor(**params_b)
model_c = xgb.XGBRegressor(**params_c)



In [18]:

model_a.fit(x_train_a, y_train_a)
model_b.fit(x_train_b, y_train_b)
model_c.fit(x_train_c, y_train_c)

KeyboardInterrupt: 

# Make predictions

In [17]:
model_a.fit(X_train[0], Y_train[0])
model_b.fit(X_train[1], Y_train[1])
model_c.fit(X_train[2], Y_train[2])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8763563332327975, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.08609213174337473, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=727, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [18]:
# Do some more stuff
y_pred_a = model_a.predict(X_frames_test[0])
y_pred_b = model_b.predict(X_frames_test[1])
y_pred_c = model_c.predict(X_frames_test[2])


# Evaluate prediction

In [ ]:

# Evaluate the model based on the validation data

mse_a = mean_squared_error(y_val_a, model_a.predict(x_val_a))
print("MSE for A: ", mse_a)
mse_b = mean_squared_error(y_val_b, model_b.predict(x_val_b))
print("MSE for B: ", mse_b)
mse_c = mean_squared_error(y_val_c, model_c.predict(x_val_c))
print("MSE for C: ", mse_c)
print("Mean MSE: ", (mse_a + mse_b + mse_c) / 3)

# Evaluate the predictions

score_a = model_a.score(x_val_a, y_val_a)
score_b = model_b.score(x_val_b, y_val_b)
score_c = model_c.score(x_val_c, y_val_c)

print("Score A: ", score_a)
print("Score B: ", score_b)
print("Score C: ", score_c)
print('')

# Get feature importance scores
models = [(model_a, 'A'), (model_b, 'B'), (model_c, 'C')]
for model in models:

    feature_importance_scores = model[0].feature_importances_

# Create a DataFrame to associate features with their importance scores
    feature_importance_df1 = pd.DataFrame({'Feature': x_train_a.columns, 'Importance': feature_importance_scores})

# Sort features by importance in descending order
    feature_importance_df1 = feature_importance_df1.sort_values(by='Importance', ascending=False)

# Print or visualize the feature importance scores
    
    print(f'Model {model[1]}')
    print(feature_importance_df1.head(10))
    print('')


# Create submission

In [19]:
y_pred = np.concatenate((y_pred_a, y_pred_b, y_pred_c), axis=0)

In [20]:
y_test_pred = y_pred

test = pd.read_csv('../data/test.csv')
test['prediction'] = y_test_pred
sample_submission = pd.read_csv('../data/sample_submission.csv')
submission = sample_submission[['id']].merge(test[['id', 'prediction']], on='id', how='left')
submission.to_csv('submission_xg_y.csv', index=False)